In [7]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [8]:
# data frames - tables with extra functionalities
df = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], columns=["A", "B", "C"], index=["X", "Y", "Z"])
df

,A,B,C
X,1,2,3
Y,4,5,6
Z,7,8,9


In [9]:
# displaying data frames

df.columns  # only column hedders
df.index    # only row indexes
df.shape    # (rows, columns)
df.size     # number of cells
df.head()   # first rows
df.tail()   # last rows
df.info()   # basic info
df.describe()   # info about content
df.nunique()    # number of unique records in each column
df["A"].unique()    # column's unique records

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, X to Z
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   A       3 non-null      int64
 1   B       3 non-null      int64
 2   C       3 non-null      int64
dtypes: int64(3)
memory usage: 96.0+ bytes


array([1, 4, 7])

In [10]:
# loading csv file
file = Path("../example-files/app_log.csv")
df = pd.read_csv(file)

In [11]:
# filtering based on row index, with ability to specifie column by their index name
df.loc[[1, 2, 5], "endpoint"]
df.loc[5:10, ["user","endpoint"]]

# filtering based on row index, with ability to specifie column by their number
df.iloc[5:10, [1, 2]]

,user,endpoint
5,edward,/login
6,celina,/logout
7,bartek,/login
8,franek,/login
9,edward,/data


In [12]:
# editing data in data frame

df.loc[[0], ["user"]] = "anna"
df.loc[[0], ["user"]]

,user
0,anna


In [13]:
# accessing single cell

df.at[0, "user"]
df.iat[0, 1]

'anna'

In [14]:
# getting value of a column

df["user"].head()
df.user.head()

0      anna
1    bartek
2    celina
3      anna
4    damian
Name: user, dtype: object

In [15]:
# sorting data (ascending by deafult)

df.sort_values(["user", "status_code"], ascending=[0, 1]).head()

,timestamp,user,endpoint,status_code
18,2025-09-28 09:04:22,franek,/data,200
30,2025-09-28 09:06:55,franek,/login,200
35,2025-09-28 09:07:55,franek,/login,200
13,2025-09-28 09:03:12,franek,/login,401
41,2025-09-28 09:09:25,franek,/login,401


In [16]:
# filtering data | rows that applys to the specified rule

df.loc[df["status_code"] == 200, ["timestamp", "user", "status_code"]]
df[df["status_code"] == 200][["timestamp", "user", "status_code"]].head()

,timestamp,user,status_code
0,2025-09-28 09:00:12,anna,200
2,2025-09-28 09:00:52,celina,200
3,2025-09-28 09:01:15,anna,200
5,2025-09-28 09:01:40,edward,200
6,2025-09-28 09:01:55,celina,200


In [17]:
# filtering data based on more than one column

df[(df["status_code"] != 200) & (df["user"] == "anna")]

,timestamp,user,endpoint,status_code
10,2025-09-28 09:02:40,anna,/data,404
28,2025-09-28 09:06:25,anna,/data,500


In [18]:
#filtering data based on string conditions

df[df["user"].str.contains("a", case=False)].head()
df[df["status_code"].astype(str).str.contains(r"^.0?.")].head()
df[df["user"].isin(["anna", "robert", "mark"])].head()

,timestamp,user,endpoint,status_code
0,2025-09-28 09:00:12,anna,/login,200
3,2025-09-28 09:01:15,anna,/data,200
10,2025-09-28 09:02:40,anna,/data,404
16,2025-09-28 09:03:50,anna,/logout,200
23,2025-09-28 09:05:22,anna,/login,200


In [19]:
# query function - differenct approach to data filtering

df.query('user == "anna" and status_code == 200')


,timestamp,user,endpoint,status_code
0,2025-09-28 09:00:12,anna,/login,200
3,2025-09-28 09:01:15,anna,/data,200
16,2025-09-28 09:03:50,anna,/logout,200
23,2025-09-28 09:05:22,anna,/login,200
34,2025-09-28 09:07:40,anna,/data,200
40,2025-09-28 09:09:10,anna,/login,200


In [20]:
# adding and removing data

df['danger_level'] = np.where(df["status_code"]==200, 5, 3) # adding new column
df.drop(columns=["timestamp"]).head()   # excluding columnt inside specific querry


,user,endpoint,status_code,danger_level
0,anna,/login,200,5
1,bartek,/login,401,3
2,celina,/login,200,5
3,anna,/data,200,5
4,damian,/login,500,3


In [21]:
# coping data frame

df_new = df.copy()

In [22]:
# renaming columns

df_new.rename(columns={'endpoint': 'action'}, inplace=True)
df_new.head()

,timestamp,user,action,status_code,danger_level
0,2025-09-28 09:00:12,anna,/login,200,5
1,2025-09-28 09:00:35,bartek,/login,401,3
2,2025-09-28 09:00:52,celina,/login,200,5
3,2025-09-28 09:01:15,anna,/data,200,5
4,2025-09-28 09:01:25,damian,/login,500,3


In [23]:
# transforming data to datetime format

df_new['timestamp'] = pd.to_datetime(df_new['timestamp'])
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   timestamp     45 non-null     datetime64[ns]
 1   user          45 non-null     object        
 2   action        45 non-null     object        
 3   status_code   45 non-null     int64         
 4   danger_level  45 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.9+ KB


In [24]:
# datetime format usecases

df_new['event_year'] = df_new['timestamp'].dt.year
df_new.head()

,timestamp,user,action,status_code,danger_level,event_year
0,2025-09-28 09:00:12,anna,/login,200,5,2025
1,2025-09-28 09:00:35,bartek,/login,401,3,2025
2,2025-09-28 09:00:52,celina,/login,200,5,2025
3,2025-09-28 09:01:15,anna,/data,200,5,2025
4,2025-09-28 09:01:25,damian,/login,500,3,2025


In [25]:
# exporting data frame to csv file

df_new.to_csv(file, index=False)    # without indexes

In [26]:
# adding / removing columns

df_new['action_category'] = df_new['action'].apply(lambda x: 'access' if (x == '/login' or x == '/logout') else 'transfer')
df_new.head()

,timestamp,user,action,status_code,danger_level,event_year,action_category
0,2025-09-28 09:00:12,anna,/login,200,5,2025,access
1,2025-09-28 09:00:35,bartek,/login,401,3,2025,access
2,2025-09-28 09:00:52,celina,/login,200,5,2025,access
3,2025-09-28 09:01:15,anna,/data,200,5,2025,transfer
4,2025-09-28 09:01:25,damian,/login,500,3,2025,access


In [27]:
def categorize_action(row):
    if (row == '/login' or row == '/logout'):
        return 'access'
    else:
        return 'transfer'

df_new['action_category'] = df_new.apply(categorize_action, axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [54]:
# aggregations - count

df.head()
df['endpoint'].value_counts()
df[df['endpoint'] == '/login']['user'].value_counts().to_dict()

{'franek': 5, 'bartek': 4, 'celina': 4, 'damian': 4, 'anna': 3, 'edward': 3}

In [6]:
# aggregations - group by

df.head()
df.groupby(['endpoint', 'status_code']).agg({'user': 'count'})
df.groupby(['user'])['endpoint'].count()

user
anna      8
bartek    7
celina    9
damian    6
edward    8
franek    7
Name: endpoint, dtype: int64

In [75]:
# aggreagations - pivot

pivot = df.pivot(columns=['user'], index='timestamp', values='status_code')
pivot.head()

user,anna,bartek,celina,damian,edward,franek
timestamp,,,,,,
2025-09-28 09:00:12,200.0,NaN,NaN,NaN,NaN,NaN
2025-09-28 09:00:35,NaN,401.0,NaN,NaN,NaN,NaN
2025-09-28 09:00:52,NaN,NaN,200.0,NaN,NaN,NaN
2025-09-28 09:01:15,200.0,NaN,NaN,NaN,NaN,NaN
2025-09-28 09:01:25,NaN,NaN,NaN,500.0,NaN,NaN
